In [1]:
import pandas as pd
import numpy as np


# Importing the fuzzywuzzy library
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from collections import OrderedDict, defaultdict

from tqdm.notebook import tqdm
from pandarallel import pandarallel
from deltalake.writer import write_deltalake


from datetime import datetime, date
from pprint import pprint

from data_analytics import utils

In [2]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
# =========================================================
data_conciliacao_convenios_datetime = date(2023, 8, 30)

this_data_virada = "2023-08-25"
global data_conciliacao_convenios

data_conciliacao_convenios = data_conciliacao_convenios_datetime.strftime("%Y-%m-%d")
data_agora = datetime.now().strftime("%Y-%m-%d")

print(f"{data_conciliacao_convenios}")

print(
    f"""
Data de apuração do faturamento: {data_conciliacao_convenios} {type(data_conciliacao_convenios)} 
Data da virada: {this_data_virada} {type(this_data_virada)}
Data agora: {data_agora} {type(data_agora)}
"""
)

2023-08-30

Data de apuração do faturamento: 2023-08-30 <class 'str'> 
Data da virada: 2023-08-25 <class 'str'>
Data agora: 2023-09-12 <class 'str'>



In [4]:
def display_options(row=7, col=7, largura=50, altura=None):
    display = pd.options.display
    display.max_columns = col
    display.max_rows = row
    display.max_colwidth = largura
    # display.width = None
    return None

### Encontre o produto mais semelhante dos dados do concorrente para um determinado produto de nossa empresa usando correspondência difusa


In [5]:
def find_most_similar_product(our_product, competitor_products):
    similarities = [
        fuzz.token_sort_ratio(our_product, competitor_product)
        for competitor_product in competitor_products
    ]
    most_similar_index = max(range(len(similarities)), key=lambda i: similarities[i])
    return most_similar_index, similarities[most_similar_index]

### TODO: eu só quero corrigir o nome do procedimento do lavite


In [6]:
df_geral = pd.read_csv(
    "data_analytics/202309121116-lavite-tabelas_convenios_procedimentos.csv", sep="#"
)
df_geral.sample(7)

,unidade,ti_id,convenio,codigo_procedimento,descricao,codigo_tiss,codigo_tuss,tabela_convenio,tipo_tabela,preco_tabela,tipo_id,status_procedimento,status_tabela,procedimento_generico,especialidade
5481,olinda,3117,SORRISO DO BEM,56020058,INSTILACAO VESICAL OU URETRAL,NaN,NaN,AMB 90,outros,40.00,1,ativo,ativa,NaN,NaN
6828,boavista,591277,GOLDEN CROSS,80120059,CONSULTA DE ESPECIALISTA EM ENDODONTIA,NaN,NaN,Tabela Golden Cross,particular,21.57,22,ativo,ativa,CONSULTA ODONTOLOGICA EM ESPECIALISTA,Diagnostico
11431,paulista,593659,Particular,86000072,Aparelho ortodôntico Fixo Autoligado,86000072,86000072,TABELA PARTICULAR ORTODONTIA,convenio,850.00,94,ativo,ativa,APARELHO FIXO AUTO-LIGADO,Ortodontia e Ortopedia Facial
14034,pina,593712,Particular,85400017,Ajuste Oclusal por acréscimo,85400017,85400017,TABELA PARTICULAR ODONTOLOGIA RESTAURADORA,convenio,56.00,94,ativo,ativa,AJUSTE OCLUSAL POR DESGASTE SELETIVO,Periodontia
2564,olinda,200,SORRISO DO BEM,25060074,REC FUNC POS OPERAT OU POS IMOBILIZACAO DE PAT...,NaN,NaN,AMB 90,outros,28.00,1,ativo,ativa,NaN,NaN
2930,olinda,566,SORRISO DO BEM,28050029,ADRENOCORTICOTROFICO (ACTH) HORMONIO (RIE),NaN,NaN,AMB 90,outros,210.00,1,ativo,ativa,NaN,NaN
8454,boavista,592998,SAUDE RECIFE - RECIPREV,83000089,EXODONTIA DE DECIDUOS,NaN,83000089,Tabela SAUDE RECIFE,particular,16.00,22,ativo,ativa,EXODONTIA DE DECIDUOS,Odontopediatria


In [ ]:
df_geral[
    ["convenio", "tabela_convenio", "tipo_tabela", "status_tabela"]
].drop_duplicates().sort_values("convenio")
df_geral[["tipo_tabela", "descricao"]][
    df_geral["descricao"].str.contains("psia")
].drop_duplicates().sort_values("tipo_tabela")

In [8]:
col = [
    "especialidade",
    "procedimento_generico",
    "descricao",
]
df_proced = df_geral[col].copy()
print(len(df_proced))

df_proced.drop_duplicates(inplace=True)
print(len(df_proced))

df_proc_generico = df_proced["procedimento_generico"].copy()
df_proc_generico.drop_duplicates(inplace=True)
print(len(df_proc_generico))

df_proc_generico.isna().sum()

df_proced_nulos = df_proced[
    df_proced["procedimento_generico"].isna() | df_proced["especialidade"].isna()
].copy()


df_proced_encontrados = df_proced[
    ~df_proced.index.isin(list(df_proced_nulos.index))
].copy()
df_proced_encontrados.sample(7)

print(
    f"Qtd procedimento nulos {len(df_proced_nulos)} | Encontrados {len(df_proced_encontrados)}"
)

14287
5039
278
Qtd procedimento nulos 3368 | Encontrados 1671


In [9]:
def ptable(df, col, txt):
    df_ = df[df[col].str.casefold().str.contains(txt)]
    return pd.pivot_table(
        df_,
        index=[
            "especialidade",
            "procedimento_generico",
            "descricao",
            # "tabela_convenio",
        ],
        values=["descricao"],
        aggfunc="count",
    )


display_options(10, 7)
ptable(df_proced_encontrados, "descricao", "rest")

In [12]:
lista_procedimentos_errados = utils.import_list("lista_procedimentos_errados")

for i in lista_procedimentos_errados:
    print(
        f"""{
            df_proced_encontrados[
                df_proced_encontrados['descricao'] == i
            ].descricao.values
        } -> {
            df_proced_encontrados[
                df_proced_encontrados['descricao'] == i
            ].procedimento_generico.values
        }"""
    )

# drop
for i in lista_procedimentos_errados:
    df_proced_encontrados.drop(
        df_proced_encontrados[df_proced_encontrados["descricao"] == i].index,
        inplace=True,
    )

# atualizando
df_proced_nulos = df_proced[
    ~df_proced.index.isin(list(df_proced_encontrados.index))
].copy()

print("\n")
print("ATUALIZACAO TABELA PROCEDIMENTOS ENCONTRADOS")

for i in lista_procedimentos_errados:
    print(
        f"""{
            df_proced_encontrados[
                df_proced_encontrados['descricao'] == i
            ].descricao.values
        } -> {
            df_proced_encontrados[
                df_proced_encontrados['descricao'] == i
            ].procedimento_generico.values
        }"""
    )

print("\n")
print("PROCEDIMENTOS ADICIONADOS NA TABELA DE PROCEDIMENTOS A CORRIGIR")

for i in lista_procedimentos_errados:
    print(
        f"""{
            df_proced_nulos[
                df_proced_nulos['descricao'] == i
            ].descricao.values
        } -> {
            df_proced_nulos[
                df_proced_nulos['descricao'] == i
            ].procedimento_generico.values
        }"""
    )

utils.export_list(lista_procedimentos_errados, "lista_procedimentos_errados")

['FIOS, PINOS, PARAFUSOS OU HASTES METALICOS INTRA-OSSEOS'] -> ['PRODUTOS: FIO DENTAL']
['FIOS OU PINOS METALICOS TRANSOSSEOS'] -> ['PRODUTOS: FIO DENTAL']
['RES. PINO'] -> ['RECONSTRUCAO DE DENTE COM PINO']


ATUALIZACAO TABELA PROCEDIMENTOS ENCONTRADOS
[] -> []
[] -> []
[] -> []


PROCEDIMENTOS ADICIONADOS NA TABELA DE PROCEDIMENTOS A CORRIGIR
['FIOS, PINOS, PARAFUSOS OU HASTES METALICOS INTRA-OSSEOS'] -> ['PRODUTOS: FIO DENTAL']
['FIOS OU PINOS METALICOS TRANSOSSEOS'] -> ['PRODUTOS: FIO DENTAL']
['RES. PINO'] -> ['RECONSTRUCAO DE DENTE COM PINO']


In [13]:
df_proced_encontrados_correcao = (
    df_proced_encontrados[
        df_proced_encontrados["procedimento_generico"]
        .str.casefold()
        .str.contains("pino")
    ]
    .sort_values(by="procedimento_generico")
    .copy()
)

df_proced_encontrados_correcao

,especialidade,procedimento_generico,descricao
71,Protese,COROA PROVISORIA COM PINO,Coroa unitária provisória c/ ou s/ pino prov. ...
2317,Protese,COROA PROVISORIA COM PINO,COROA UNITARIA PROVISORIA COM OU SEM PINO PROV...
6523,Protese,COROA PROVISORIA COM PINO,COROA PROVISORIA COM PINO
10675,Protese,COROA PROVISORIA COM PINO,COR PROV C/PINO
1936,Protese,COROA PROVISORIA COM PINO,COROA PROV C/ PINO
...,...,...,...
1702,Dentistica,RESTAURACAO COM PINO,RESTAURAÇÃO A PINO INTRADENTINARIO
1300,Dentistica,RESTAURACAO COM PINO,RESTAURAÇÃO A PINO
181,Dentistica,RESTAURACAO COM PINO,Restauração a pino intra-dentinário
5824,Dentistica,RESTAURACAO COM PINO,Restauração com Pino


Remover errados

In [14]:
df_proced_encontrados["procedimento_generico"].to_list()

['CONSULTA ODONTOLOGICA',
 'URGENCIA ODONTOLOGICA',
 'URGENCIA ODONTOLOGICA',
 'PULPOTOMIA',
 'RECIMENTACAO DE TRABALHOS PROTETICOS',
 'TRATAMENTO DE ALVEOLITE',
 'COLAGEM DE FRAGMENTOS DENTARIOS',
 'CONTROLE DE HEMORRAGIA',
 'INCISAO E DRENAGEM EXTRA ORAL DE ABSCESSO, HEMATOMA E/OU FLEGMAO DA REGIAO BUCO-MAXILO-FACIAL',
 'INCISAO E DRENAGEM INTRA ORAL DE ABSCESSO, HEMATOMA E/OU FLEGMAO DA REGIAO BUCO-MAXILO-FACIAL',
 'REIMPLANTE DENTARIO COM CONTENCAO',
 'CURATIVO DE DEMORA',
 'APLICACAO DE SELANTE DE FOSSULAS E FISSURAS',
 'APLICACAO DE CARIOSTATICO',
 'ADEQUACAO DO MEIO BUCAL',
 'REMINERALIZACAO',
 'TESTE DE FLUXO SALIVAR',
 'CONDICIONAMENTO EM ODONTOLOGIA',
 'COROA DE ACETATO EM DENTE DECIDUO',
 'EXODONTIA DE DECIDUOS',
 'PULPOTOMIA',
 'TRATAMENTO ENDODONTICO EM DENTE DECIDUO',
 'EXODONTIA DE PERMANENTE',
 'EXODONTIA DE RAIZ RESIDUAL',
 'EXODONTIA A RETALHO',
 'ODONTO-SECCAO',
 'ALVEOLOPLASTIA',
 'ALVEOLOPLASTIA',
 'FRENULECTOMIA',
 'FRENULECTOMIA',
 'EXODONTIA DE SEMI-INCLUSOS / I

In [ ]:
# folder
pickle_path = "../ciadosorriso_metabase/data/pickle/"
csv_path = "data/db/csv/"
filename_pkl = f"{data_conciliacao_convenios}_database_panda_engine_atendimento_itens_generico_completo.pkl"
file_path_ati = f"{pickle_path}{filename_pkl}"
if os.path.exists(file_path_ati):
    print(f"\n🟢 Arquivo {file_path_ati} encontrado.")
    df_original_ = pd.read_pickle(file_path_ati)
else:
    print(f"\n⛔️ Arquivo {file_path_ati} não encontrado.")

In [22]:
df_geral["procedimento_generico"]

0                         CONSULTA ODONTOLOGICA
1                         URGENCIA ODONTOLOGICA
2                         URGENCIA ODONTOLOGICA
3                                    PULPOTOMIA
4          RECIMENTACAO DE TRABALHOS PROTETICOS
                          ...                  
14282    AVALIACAO ODONTOLOGICA ESTETICA FACIAL
14283        AVALIACAO ODONTOLOGICA PERIODONTIA
14284               AVALIACAO ODONTOLOGICA KIDS
14285                                       NaN
14286                                       NaN
Name: procedimento_generico, Length: 14287, dtype: object

In [30]:
display_options()
display(
    df_geral[df_geral["descricao"].str.casefold().str.contains("psia")][
        ["descricao", "procedimento_generico"]
    ].drop_duplicates()
)

,descricao,procedimento_generico
41,Biópsia de Boca,BIOPSIA
42,Biópsia de Glândula Salivar,BIOPSIA
43,Biópsia de Lábio,BIOPSIA
...,...,...
5631,BIOPSIA DE EPIDIDIMO,BIOPSIA
6246,BIOPSIA DE GLANDULA SALIVAR,BIOPSIA
9027,OMOPATOLOGICO DE MATERIAL DE BIOPSIA BUCO MAXI...,DIAGNOSTICO ANATOMOPATOLOGICO EM MATERIAL DE B...


In [35]:
len(competitor_products)

5032

In [31]:
# Sample data from our company and competitors

lista_proced_generico_adicionar = [
    "BIOPSIA DE BOCA"
    "BIOPSIA DE LABIO"
    "BIOPSIA DE LINGUA"
    "BIOPSIA DE MANDIBULA"
    "BIOPSIA DE MAXILA"
    "BIOPSIA DE GLANDULA SALIVAR"
]

our_products = (
    df_proced_encontrados["procedimento_generico"].drop_duplicates().to_list()
) + lista_proced_generico_adicionar

competitor_products = df_geral["descricao"].drop_duplicates().to_list()

# Mapping our products with competitor products using fuzzy matching
mapped_products = {}
for our_product in our_products:
    best_match_index, similarity_score = find_most_similar_product(
        our_product, competitor_products
    )

    mapped_products[our_product] = (
        competitor_products[best_match_index],
        similarity_score,
    )

# Displaying the mapped products
print("Product Mapping:")
for our_product, (competitor_product, similarity_score) in mapped_products.items():
    print(f"{our_product} => {competitor_product} (Similarity: {similarity_score})")

Product Mapping:
CONSULTA ODONTOLOGICA => CONSULTA ODONTOLOGICA (Similarity: 100)
URGENCIA ODONTOLOGICA => CONSULTA ODONTOLOGICA - URGENCIA (Similarity: 82)
PULPOTOMIA => Pulpotomia (Similarity: 100)
RECIMENTACAO DE TRABALHOS PROTETICOS => RECIMENTAÇÃO DE TRABALHOS PROTÉTICOS (Similarity: 96)
TRATAMENTO DE ALVEOLITE => Tratamento de Alveolite (Similarity: 100)
COLAGEM DE FRAGMENTOS DENTARIOS => Colagem de fragmentos dentários (Similarity: 98)
CONTROLE DE HEMORRAGIA => HEMACIAS CONTAGEM DE (Similarity: 71)
INCISAO E DRENAGEM EXTRA ORAL DE ABSCESSO, HEMATOMA E/OU FLEGMAO DA REGIAO BUCO-MAXILO-FACIAL => Incisão e Drenagem extra-oral de abscesso, hematoma e/ou flegmão da
região buco-maxilo-facial (Similarity: 98)
INCISAO E DRENAGEM INTRA ORAL DE ABSCESSO, HEMATOMA E/OU FLEGMAO DA REGIAO BUCO-MAXILO-FACIAL => Incisão e Drenagem intra-oral de abscesso, hematoma e/ou flegmão da região buco-maxilo-facial (Similarity: 98)
REIMPLANTE DENTARIO COM CONTENCAO => REIMPLANTE DENTARIO COM CONTENÇAO (S

In [33]:
mapped_products

{'CONSULTA ODONTOLOGICA': ('CONSULTA ODONTOLOGICA', 100),
 'URGENCIA ODONTOLOGICA': ('CONSULTA ODONTOLOGICA - URGENCIA', 82),
 'PULPOTOMIA': ('Pulpotomia', 100),
 'RECIMENTACAO DE TRABALHOS PROTETICOS': ('RECIMENTAÇÃO DE TRABALHOS PROTÉTICOS',
  96),
 'TRATAMENTO DE ALVEOLITE': ('Tratamento de Alveolite', 100),
 'COLAGEM DE FRAGMENTOS DENTARIOS': ('Colagem de fragmentos dentários', 98),
 'CONTROLE DE HEMORRAGIA': ('HEMACIAS CONTAGEM DE', 71),
 'INCISAO E DRENAGEM EXTRA ORAL DE ABSCESSO, HEMATOMA E/OU FLEGMAO DA REGIAO BUCO-MAXILO-FACIAL': ('Incisão e Drenagem extra-oral de abscesso, hematoma e/ou flegmão da\nregião buco-maxilo-facial',
  98),
 'INCISAO E DRENAGEM INTRA ORAL DE ABSCESSO, HEMATOMA E/OU FLEGMAO DA REGIAO BUCO-MAXILO-FACIAL': ('Incisão e Drenagem intra-oral de abscesso, hematoma e/ou flegmão da região buco-maxilo-facial',
  98),
 'REIMPLANTE DENTARIO COM CONTENCAO': ('REIMPLANTE DENTARIO COM CONTENÇAO',
  98),
 'CURATIVO DE DEMORA': ('EMER-CURATIVO', 77),
 'APLICACAO DE SE

In [ ]:
# Sample data from our company and competitors
# our_products = ["iPhone X", "iPad Pro", "MacBook Air", "Apple Watch", "AirPods"]
# competitor_products = ["iFone X", "iTablet Pro", "Mac Air", "iWatch", "Earbuds"]

# Mapping our products with competitor products using fuzzy matching
# mapped_products = {}
# for our_product in our_products:
#     best_match_index, similarity_score = find_most_similar_product(
#         our_product, competitor_products
#     )

#     mapped_products[our_product] = (
#         competitor_products[best_match_index],
#         similarity_score,
#     )

# # Displaying the mapped products
# print("Product Mapping:")
# for our_product, (competitor_product, similarity_score) in mapped_products.items():
#     print(f"{our_product} => {competitor_product} (Similarity: {similarity_score})")

# ------------------------------------------------------------------------------------------------------------------------

In [ ]:
def compare_strings(a, b):
    return fuzz.token_sort_ratio(a, b)


def fuzzy_procedimento_generico(df, threshold=90):
    duplicates = set()
    processed = []

    for i, row in df.iterrows():
        if i not in duplicates:
            processed.append(row)

            for j, other_row in df.iterrows():
                if i != j and j not in duplicates:
                    score = compare_strings(
                        row["descricao"], other_row["procedimento_generico"]
                    )

                    if score >= threshold:
                        duplicates.add(j)

    return pd.DataFrame(processed)


fuzzy_procedimento_generico(df, threshold=80)

In [ ]:
def return_results(list_of_dicts, query, threshold):
    scores = []
    for index, item in enumerate(list_of_dicts):
        values = list(item.values())
        ratios = [
            fuzz.ratio(str(query), str(value)) for value in values
        ]  # ensure both are in string
        scores.append({"index": index, "score": max(ratios)})

    filtered_scores = [item for item in scores if item["score"] >= threshold]
    sorted_filtered_scores = sorted(
        filtered_scores, key=lambda k: k["score"], reverse=True
    )
    filtered_list_of_dicts = [
        list_of_dicts[item["index"]] for item in sorted_filtered_scores
    ]
    return filtered_list_of_dicts

In [ ]:
# df.drop_duplicates(')